In [1]:
#tipicas
import pandas as pd
import numpy as np

#para la creacion de carpetas
import os
import shutil

#para el test-train split
import splitfolders #(necesita pip install split-folders)

#datagen
from keras.preprocessing.image import ImageDataGenerator

#modelos
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras.callbacks import EarlyStopping

2023-01-15 10:28:29.408851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 10:28:30.826528: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-15 10:28:30.826576: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-15 10:28:40.555169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

### primero tenemos q separar las imagenes en carpetas segun su master class

In [2]:
#importamos el df con las labels
path_csv = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/df1.csv"
df_styles = pd.read_csv(path_csv, error_bad_lines = False)

/tmp/ipykernel_3967/3561016237.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_styles = pd.read_csv(path_csv, error_bad_lines = False)


### creamos los filenames 

In [4]:
#guardamos la lista de las labels
labels= np.unique(df_styles.masterCategory).tolist()

#guardamos la lista de los ids 
list_id = list(df_styles.id)

In [5]:
#en files guardamos los path a cada foto en files y el nombre con .jpg en images
images = []
for file in list_id:
    images.append(str(file) + ".jpg")
    
path_img = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/df1"
files = []
for i in images:
    files.append((os.path.join(path_img, i)))

### en un df guardamos el id, master category y agregamos el nombre de la imagen con jpg

In [8]:
df = df_styles.drop(df_styles.columns.difference(['id', 'masterCategory']),1)
df['name'] = images

/tmp/ipykernel_5408/1087487744.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df_styles.drop(df_styles.columns.difference(['id', 'masterCategory']),1)


### creamos una variable con los nombres de las clases 

In [9]:
classes = df.sort_values('masterCategory')
class_names = np.unique(df_styles.masterCategory).tolist()

### esto crea las carpetas vacias segun master category

In [79]:
for i in class_names:
    os.makedirs(os.path.join("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data",i))

In [80]:
for i in class_names:     # I ->  class label names
    for c in list(df[df['masterCategory']== i]['name']):    # c  ->  individual image 
    # Creating path to the image
        get_image = os.path.join("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/df1",str(c))
        if not os.path.exists("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data/"+i+"/"+c):
            move = shutil.copy(get_image,"/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data/"+i)

### ahora hacemos la division de train test de las carpetas

In [13]:
#primero creamos la carpeta donde estara nuestra data spliteada
# os.makedirs("/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split")

In [43]:
input_folder = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data/"
output_folder = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split"

In [45]:
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output_folder,
                    seed=42,
                    ratio=(.7, .2, .1), #usamos 70% train, 20% val, 10% test 
                    group_prefix=None)

### ahora que tenemos en subcarpetas todo probamos el image data gen

In [7]:
# iniciamos el gen escalando para que el input este entre 0 y 1
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#creamos los generadores con batch de 30 fotos con el dir correpondiente
train_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split/train"
val_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split/val"
test_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split/test"

train_generator = train_datagen.flow_from_directory(
                                train_dir,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

val_generator = train_datagen.flow_from_directory(
                                val_dir,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

test_generator = train_datagen.flow_from_directory(
                                test_dir,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

Found 13997 images belonging to 7 classes.
Found 3998 images belonging to 7 classes.
Found 2005 images belonging to 7 classes.


### ahora creamos el modelo 1 para master category

In [8]:
def load_own_model():
    model = Sequential()
    model.add(layers.Input(shape=(480,360,3), name='input'))
    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))

    model.compile(loss = 'categorical_crossentropy', 
                    optimizer = 'adam',
                    metrics = 'accuracy')
    
    return model

In [9]:
model = load_own_model()
model.summary()

2023-01-14 16:24:30.185909: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-14 16:24:30.187269: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-14 16:24:30.188216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-HHE71063): /proc/driver/nvidia/version does not exist
2023-01-14 16:24:30.191014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 471, 351, 16)      4816      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 157, 117, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 110, 32)      32800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 31, 32)        36896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 10, 32)       0

In [11]:
history = model.fit_generator(
             train_generator,
             steps_per_epoch = 15,
             epochs = 30,
             callbacks = EarlyStopping(patience = 5, restore_best_weights=True),
             validation_data = val_generator,
             validation_steps = 15)

/tmp/ipykernel_21765/2842717622.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
15/15 [==============================] - 63s 4s/step - loss: 1.1176 - accuracy: 0.5600 - val_loss: 1.1697 - val_accuracy: 0.4680
Epoch 2/30
15/15 [==============================] - 58s 4s/step - loss: 1.0920 - accuracy: 0.5200 - val_loss: 1.0160 - val_accuracy: 0.4920
Epoch 3/30
15/15 [==============================] - 62s 4s/step - loss: 0.9587 - accuracy: 0.6667 - val_loss: 0.7936 - val_accuracy: 0.7120
Epoch 4/30
15/15 [==============================] - 71s 5s/step - loss: 0.8238 - accuracy: 0.7600 - val_loss: 0.5606 - val_accuracy: 0.7920
Epoch 5/30
15/15 [==============================] - 70s 5s/step - loss: 0.6407 - accuracy: 0.8533 - val_loss: 0.5868 - val_accuracy: 0.8040
Epoch 6/30
15/15 [==============================] - 59s 4s/step - loss: 0.5843 - accuracy: 0.7467 - val_loss: 0.9110 - val_accuracy: 0.6280
Epoch 7/30
15/15 [==============================] - 56s 4s/step - loss: 0.8882 - accuracy: 0.6667 - val_loss: 0.7313 - val_accuracy: 0.7520
Epoch 8/30
15/15 [==

In [12]:
model.evaluate_generator(val_generator,verbose = 0)

/tmp/ipykernel_21765/619196297.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(val_generator,verbose =0)


[0.4457930326461792, 0.8611806035041809]

In [13]:
model.evaluate_generator(test_generator,verbose = 0)

/tmp/ipykernel_21765/984775710.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator,verbose = 0)


[0.4306773245334625, 0.8673316836357117]

### probamos lo mismo pero eliminando free items y sporting goods


In [21]:
input_folder_sin = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/data_sin/"
output_folder_sin = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split_sin"

In [22]:
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder_sin, output=output_folder_sin,
                    seed=42,
                    ratio=(.7, .2, .1), #usamos 70% train, 20% val, 10% test 
                    group_prefix=None)

In [23]:
train_datagen_sin = ImageDataGenerator(rescale = 1./255)
val_datagen_sin = ImageDataGenerator(rescale = 1./255)
test_datagen_sin = ImageDataGenerator(rescale=1./255)

#creamos los generadores con batch de 30 fotos con el dir correpondiente
train_dir_sin= "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split_sin/train"
val_dir_sin = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split_sin/val"
test_dir_sin= "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/split_sin/test"

train_generator_sin = train_datagen_sin.flow_from_directory(
                                train_dir_sin,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

val_generator_sin = val_datagen_sin.flow_from_directory(
                                val_dir_sin,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

test_generator_sin = test_datagen_sin.flow_from_directory(
                                test_dir_sin,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

Found 13963 images belonging to 5 classes.
Found 3989 images belonging to 5 classes.
Found 1998 images belonging to 5 classes.


In [25]:
def load_own_model_sin():
    model = Sequential()
    model.add(layers.Input(shape=(480,360,3), name='input'))
    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))

    model.compile(loss = 'categorical_crossentropy', 
                    optimizer = 'adam',
                    metrics = 'accuracy')
    
    return model

In [26]:
model_sin = load_own_model_sin()
history = model_sin.fit_generator(
             train_generator_sin,
             steps_per_epoch = 15,
             epochs = 30,
             callbacks = EarlyStopping(patience = 5, restore_best_weights=True),
             validation_data = val_generator_sin,
             validation_steps = 15)

Epoch 1/30


/tmp/ipykernel_21765/3258402138.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model_sin.fit_generator(


15/15 [==============================] - 28s 2s/step - loss: 1.4633 - accuracy: 0.3733 - val_loss: 1.2280 - val_accuracy: 0.5333
Epoch 2/30
15/15 [==============================] - 23s 1s/step - loss: 1.1803 - accuracy: 0.5467 - val_loss: 1.1067 - val_accuracy: 0.6267
Epoch 3/30
15/15 [==============================] - 23s 2s/step - loss: 0.9484 - accuracy: 0.6267 - val_loss: 0.6732 - val_accuracy: 0.7733
Epoch 4/30
15/15 [==============================] - 22s 1s/step - loss: 0.7573 - accuracy: 0.6933 - val_loss: 0.7530 - val_accuracy: 0.7200
Epoch 5/30
15/15 [==============================] - 21s 1s/step - loss: 0.9608 - accuracy: 0.5733 - val_loss: 0.9499 - val_accuracy: 0.6133
Epoch 6/30
15/15 [==============================] - 23s 2s/step - loss: 0.5592 - accuracy: 0.8000 - val_loss: 0.5638 - val_accuracy: 0.7867
Epoch 7/30
15/15 [==============================] - 23s 2s/step - loss: 0.7454 - accuracy: 0.7067 - val_loss: 0.5618 - val_accuracy: 0.8267
Epoch 8/30
15/15 [=============

In [27]:
model_sin.evaluate_generator(val_generator_sin,verbose = 0)

/tmp/ipykernel_21765/2702451891.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_sin.evaluate_generator(val_generator_sin,verbose = 0)


[0.34655869007110596, 0.8846828937530518]

In [28]:
model_sin.evaluate_generator(test_generator_sin,verbose = 0)

/tmp/ipykernel_21765/2449133228.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_sin.evaluate_generator(test_generator_sin,verbose = 0)


[0.34003281593322754, 0.8823823928833008]